In [1]:
# procedure for calculating hydraulic confinement along a pressure tunnel 
# completed procedure will in the form of a design report
#   safety factor against hydraulic confinement calculated at stationed points along tunnel alignment
#   calculation determines the minimum dstance from the stationed point to the terrain surface 

# project name: 'Test'

In [2]:
# ToDo
#   put common functions (e.g. setup, data format conversions) into a separate module/API
#   use Nam Ang HEP data for initial testing and demo
#   provide full desgin criteria text in Jupyter Notebook design report

In [109]:
# import required python libraires
import numpy as np
from numpy import *
import pandas as pd
import geopandas as gpd
import shapely as sp
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.plotly as py
#import gmaps as gmaps
#import plotly.graph_objs as go

# testing
#import random
# check pd and gpd version
#print(pd.__version__)
#print(gpd.__version__)

# set up plotly in 'offline' mode
# this should be documented

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
print (__version__) # requires version >= 1.9.0

# collect all plotly setups here

import plotly.offline as plotly  #testing

from plotly.graph_objs import *  #testing

ImportError: No module named gmaps

In [4]:
# python setup for qgis processing
import sys, os
from qgis.core import QgsApplication
from PyQt4.QtGui import QApplication
app = QApplication([], True)  #True -> window display enabled
QgsApplication.setPrefixPath("/usr", True)
QgsApplication.initQgis()
sys.path.append('/usr/share/qgis/python/plugins')  #export PYTHONPATH not needed in start script
from processing.core.Processing import Processing
Processing.initialize() 
import processing
#from processing.tools import *  #not needed currently

In [5]:
# import DSS modules
#import geospatial as geo

In [6]:
# set wd for this procedure and project 
os.chdir("/home/kaelin_joseph/DSS.HydraulicConfinement/")

In [7]:
# define required input files
DTM = "data/swissALTI3D_.tif"  #DEM with surface topography
Alignment = "data/TestAlignment.p0.csv"  #tunnel alignment
#   Alignment contains fields "Point","Type","Station","Northing","Easting","Elevation"
#   check Alignment data: no trailing blank lines, no duplicate lines
Slope = "data/TestTerrainSlope.tif"  #DEM containing slope angle as attribute as new Notebook            #ToDo KLK

In [8]:
# define required inout data
crs = {'init': 'epsg:2056'}  #define crs for project
grass_region = "2603510,2624270,1260650,1274890"  #map region
grass_station_dist = 50  #target station interval
c = 0.5  #ring buffer radius = c*h  (h=overburden)
res = 20.0  #nominal ring buffer grid resolution 

In [9]:
# define temporary data
Alignment_shp ='tmp/TestAlignment.shp'  #alignment shp from Alignment
Alignment_grass_csv = 'tmp/TestAlignmentGrass.csv'  #alignment csv fixed for grass
Alignment_line_shp = "tmp/TestAlignmentLine.shp"  #intermediate data
Alignment_stationed_shp = "tmp/TestAlignmentStationed.shp"  #alignment shp containing station points
Alignment_dtm_csv = "tmp/TestAlignmentDTM.csv"  #alignment including terrain elevations at station points
Buffer_shp = "tmp/TestBuffer.shp"  #buffer shp containing ring grid points at a particular station point
Buffer_all_csv = "tmp/TestBufferAll.csv"  # all station point ring buffers written to csv
Buffer_all_shp = "tmp/TestBufferFinal.shp"
Buffer_dtm_csv = "tmp/BufferDTM.csv"

In [10]:
# define output files


In [11]:
# create alignment_df (dataframe) from Alignment csv                         #ToDo JK: make into csv -> df function
alignment_df = pd.read_csv(Alignment)

# delete row if only NA are present in row
alignment_df = alignment_df.dropna(how = "all")
# round alignment_df to three decimals
alignment_df = alignment_df.round(decimals=3)

In [12]:
# create stationed alignment as Alignment_stationed_shp                     #ToDo JK: make into stationing function
# from points in alignment_df
#   required grass input data: x, y coordinates at each alignment point
#   grass input for function v.in.line 1) must have x first and y second; 2) no header 
#   grass output: x, y coordinates at each station point along alignment
alignment_df_grass = alignment_df.loc[:,["Easting", "Northing"]]  #x first and y second
# write Alignment_grass_csv
alignment_df_grass.to_csv(Alignment_grass_csv, header=False, index=False)  #no header

# points to line, write output to Alignment_line_shp
processing.runalg("grass7:v.in.lines",Alignment_grass_csv,",",False,
                  grass_region,0,Alignment_line_shp)  #no spaces between commas

# line to station points, ouput segmented polyline to Alignment_stationed_shp
processing.runalg("grass7:v.to.points",Alignment_line_shp,grass_station_dist,1,True,
                  grass_region,-1,0.0001,0,Alignment_stationed_shp)  #no spaces between commas

{'output': 'tmp/TestAlignmentStationed.shp'}

In [13]:
# create alignment_stationed_df from Alignment_stationed_shp                #ToDo JK: make into shp -> df function
#   required output data: x_align, y_align at each station point
alignment_stationed_df = gpd.read_file(Alignment_stationed_shp)

# create columns for x_align, y_align, then delete columns cat_ and geometry
alignment_stationed_df["x_align"] = alignment_stationed_df.geometry.x
alignment_stationed_df["y_align"] = alignment_stationed_df.geometry.y
alignment_stationed_df = alignment_stationed_df.drop(columns =["cat_", "geometry"])

In [14]:
# add required fields to alignment_stationed_df

# add "id" 
alignment_stationed_df["id_point"] = alignment_stationed_df.index

# add "distance_stat"
alignment_stationed_df["distance_stat"] = np.nan
for n in range(0, len(alignment_stationed_df)-1):
    alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("distance_stat")] = (
        ((alignment_stationed_df.iloc[n +1]["x_align"] - alignment_stationed_df.iloc[n]["x_align"])**2
        +(alignment_stationed_df.iloc[n +1]["y_align"] - alignment_stationed_df.iloc[n]["y_align"])**2 )**(0.5) )

# add "distance_stat_sum"
alignment_stationed_df["distance_stat_sum"] = np.nan
for n in range(0, len(alignment_stationed_df) -1):
    distance = ( alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(0,n +1))), 
                        "distance_stat"] )
    distances = distance.tolist()
    alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("distance_stat_sum")] = (
                                                                                        sum(distances) )
    
alignment_stationed_df.head()

,x_align,y_align,id_point,distance_stat,distance_stat_sum
0,2.612072e+06,1269294.122,0,49.171662,49.171662
1,2.612111e+06,1269264.730,1,49.171662,98.343324
2,2.612150e+06,1269235.338,2,49.171662,147.514986
3,2.612190e+06,1269205.946,3,49.171662,196.686648
4,2.612229e+06,1269176.554,4,49.171662,245.858310


In [15]:
# add required field "distance_intermed_align" to alignment_df
alignment_df["distance_intermed_align"] = np.nan

for n in range(0, len(alignment_df) -1):
    alignment_df.iloc[n, alignment_df.columns.get_loc("distance_intermed_align")] = (
        ((alignment_df.iloc[n +1]["Easting"]-alignment_df.iloc[n]["Easting"])**2 
             +(alignment_df.iloc[n +1]["Northing"]-alignment_df.iloc[n]["Northing"])**2 )**(0.5) )
    
alignment_df.head()  

,Point,Type,Station,Northing,Easting,Elevation,distance_intermed_align
0,0,NaN,204+874.1,1269294.122,2612071.548,220.962,442.544958
1,221,PT,205+389.861,1269029.594,2612426.331,203.058,3736.888722
2,EC10:3,EC,209+528.128,1265960.779,2614558.633,218.579,NaN


In [16]:
# join alignment_df to alignment_stationed_df
alignment_stationed_df = pd.merge(left= alignment_stationed_df, right = alignment_df, 
                 left_on = ["x_align","y_align"], 
                 right_on = ["Easting","Northing"], how = "left")

# clean up alignment_stationed_df
try:
    alignment_stationed_df = (
        alignment_stationed_df.drop(columns =["Point", "Type", "Northing", "Easting"]) )
except:
    pass

alignment_stationed_df.head()

,x_align,y_align,id_point,distance_stat,distance_stat_sum,Station,Elevation,distance_intermed_align
0,2.612072e+06,1269294.122,0,49.171662,49.171662,204+874.1,220.962,442.544958
1,2.612111e+06,1269264.730,1,49.171662,98.343324,NaN,NaN,NaN
2,2.612150e+06,1269235.338,2,49.171662,147.514986,NaN,NaN,NaN
3,2.612190e+06,1269205.946,3,49.171662,196.686648,NaN,NaN,NaN
4,2.612229e+06,1269176.554,4,49.171662,245.858310,NaN,NaN,NaN


In [17]:
# get id_points for alignment points in alignment_stationed_df 
#   select points where Elevation of point not NaN
id_points_align =  (
    alignment_stationed_df.loc[(alignment_stationed_df.Elevation.isin(alignment_df["Elevation"])), "id_point"] )
id_points_align= id_points_align.tolist()
id_points_align                                                                         

[0, 9, 84]

In [18]:
# prepare intermediated data in alignment_stationed_df required to interpolate alignment elevations

# fill in "Elevation" and "distance_intermed_align" for points in alignment_stationed_df 
#   where points of alignment_points_df != alignment_df

for n in range(0, len(id_points_align) -1): 
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n] +1, 
                                id_points_align[n +1]))), "Elevation"] = ( 
                                                                    alignment_df["Elevation"][n] )

for n in range(0, len(id_points_align) -1): 
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n] +1, 
                                id_points_align[n +1]))), "distance_intermed_align"] = ( 
                                                        alignment_df["distance_intermed_align"][n] )

# add "distance_intermed_stat" to alignment_stationed_df 
alignment_stationed_df["distance_intermed_stat"] = np.nan

for n in range(0, 1): 
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n], 
                                id_points_align[n +1]))), "distance_intermed_stat"] =  ( 
                                                    alignment_stationed_df["distance_stat_sum"] )
for n in range(1, len(id_points_align) -1): 
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n], 
                                id_points_align[n +1]))), "distance_intermed_stat"] = ( 
                                                    alignment_stationed_df["distance_stat_sum"] - 
                                                    alignment_df["distance_intermed_align"][n -1] )
    
alignment_stationed_df.head()

,x_align,y_align,id_point,distance_stat,distance_stat_sum,Station,Elevation,distance_intermed_align,distance_intermed_stat
0,2.612072e+06,1269294.122,0,49.171662,49.171662,204+874.1,220.962,442.544958,49.171662
1,2.612111e+06,1269264.730,1,49.171662,98.343324,NaN,220.962,442.544958,98.343324
2,2.612150e+06,1269235.338,2,49.171662,147.514986,NaN,220.962,442.544958,147.514986
3,2.612190e+06,1269205.946,3,49.171662,196.686648,NaN,220.962,442.544958,196.686648
4,2.612229e+06,1269176.554,4,49.171662,245.858310,NaN,220.962,442.544958,245.858310


In [19]:
# interpolate alignment elevation ("z_align") at all station points and write to alignment_stationed_df

# add variable "z_align" to alignment_stationed_df
alignment_stationed_df["z_align"] = np.nan

for i in range(0, len(alignment_stationed_df)):
    # alignment points
    if i in id_points_align:
        alignment_stationed_df.iloc[i, alignment_stationed_df.columns.get_loc("z_align")] = ( 
                                                        alignment_stationed_df.iloc[i]["Elevation"] )
    # stationed points
    else:
        id_points_align_plus_point_n = id_points_align
        id_points_align_plus_point_n.append(i)
        id_points_align_plus_point_n.sort()
        
        m = id_points_align_plus_point_n.index(i) +1  #index of point n  +1  -> next alignment point
        n = id_points_align_plus_point_n[m]  #id_point of next alignment point  
        o = id_points_align_plus_point_n.index(i) -1  #index of point n  -1  -> previous alignment point
        p = id_points_align_plus_point_n[o]  #id_point of previous alignment point
        
        alignment_stationed_df.iloc[i, alignment_stationed_df.columns.get_loc("z_align")] = ( 
                                                    alignment_stationed_df.iloc[p]["Elevation"] 
                                                    +(alignment_stationed_df.iloc[n]["Elevation"] 
                                                    -alignment_stationed_df.iloc[p]["Elevation"]) 
                                                        /alignment_stationed_df.iloc[i]["distance_intermed_align"]
                                                        *alignment_stationed_df.iloc[i-1]["distance_intermed_stat"] )

        id_points_align_plus_point_n.remove(i)

alignment_stationed_df = alignment_stationed_df.drop(columns = ["distance_intermed_align"])
alignment_stationed_df = alignment_stationed_df.drop(columns = ["distance_intermed_stat"])
alignment_stationed_df = alignment_stationed_df.drop(columns = ["Elevation"])
alignment_stationed_df.head()

,x_align,y_align,id_point,distance_stat,distance_stat_sum,Station,z_align
0,2.612072e+06,1269294.122,0,49.171662,49.171662,204+874.1,220.962000
1,2.612111e+06,1269264.730,1,49.171662,98.343324,NaN,218.972667
2,2.612150e+06,1269235.338,2,49.171662,147.514986,NaN,216.983333
3,2.612190e+06,1269205.946,3,49.171662,196.686648,NaN,214.994000
4,2.612229e+06,1269176.554,4,49.171662,245.858310,NaN,213.004667


In [20]:
# add required field "z_dtm_align" to alignment_stationed_df 

# list of shapely geometry points                                                #ToDo JK: make df -> shp function           
alignment_stationed_geometry = ( 
    [sp.geometry.Point(row['x_align'], row['y_align']) for key, row in alignment_stationed_df.iterrows()] )
# create alignment_stationed_geometry_df
alignment_stationed_geometry_df = ( 
    gpd.GeoDataFrame(alignment_stationed_df, geometry=alignment_stationed_geometry, crs = crs) )
# write df to Alignment_stationed_shp (overwrite file)
alignment_stationed_geometry_df.to_file(Alignment_stationed_shp, driver='ESRI Shapefile') 

# get DTM values for alignment_points                                     #ToDo JK: make into what.points function
#   write to Alignment_dtm_csv
processing.runalg("grass7:r.what.points",DTM,Alignment_stationed_shp, "NA",",", 500,
                  True,False,False,False,False,grass_region,-1,0.0001,Alignment_dtm_csv)

# create alignment_dtm_df (dataframe) from Alignment_dtm_csv 
alignment_dtm_df = pd.read_csv(Alignment_dtm_csv)

# rename col=tmp... to "z_dtm_align"
alignment_dtm_df_col_tmp = [col for col in alignment_dtm_df.columns if 'tmp' in col]
if len(alignment_dtm_df_col_tmp) != 1:
    print "Extraction of DTM col=tmp did not work properly for alignment. Please check"
    exit()
alignment_dtm_df = alignment_dtm_df.rename(
    columns= {alignment_dtm_df_col_tmp[0]: "z_dtm_align"})

# write alignment_dtm_df["z_dtm_align"] to alignment_stationed_df["z_dtm_align"]
alignment_stationed_df["z_dtm_align"] = alignment_dtm_df["z_dtm_align"]

alignment_stationed_df = alignment_stationed_df.drop(columns = ["geometry"])
alignment_stationed_df.head()

This can cause unexpected results.


,x_align,y_align,id_point,distance_stat,distance_stat_sum,Station,z_align,z_dtm_align
0,2.612072e+06,1269294.122,0,49.171662,49.171662,204+874.1,220.962000,253.5240
1,2.612111e+06,1269264.730,1,49.171662,98.343324,NaN,218.972667,257.3883
2,2.612150e+06,1269235.338,2,49.171662,147.514986,NaN,216.983333,257.0707
3,2.612190e+06,1269205.946,3,49.171662,196.686648,NaN,214.994000,256.3565
4,2.612229e+06,1269176.554,4,49.171662,245.858310,NaN,213.004667,255.8041


In [21]:
# Add require field "h" to alignment_stationed_df = overburden depth above station point 
alignment_stationed_df["h"] = alignment_stationed_df["z_dtm_align"] - alignment_stationed_df["z_align"] 
alignment_stationed_df.head()

,x_align,y_align,id_point,distance_stat,distance_stat_sum,Station,z_align,z_dtm_align,h
0,2.612072e+06,1269294.122,0,49.171662,49.171662,204+874.1,220.962000,253.5240,32.562000
1,2.612111e+06,1269264.730,1,49.171662,98.343324,NaN,218.972667,257.3883,38.415633
2,2.612150e+06,1269235.338,2,49.171662,147.514986,NaN,216.983333,257.0707,40.087367
3,2.612190e+06,1269205.946,3,49.171662,196.686648,NaN,214.994000,256.3565,41.362500
4,2.612229e+06,1269176.554,4,49.171662,245.858310,NaN,213.004667,255.8041,42.799433


In [22]:
# define make_buffer to get buffer grid points at all station points along alignment
def make_buffer(point, overburden):
    h = overburden
    if h < 0.0:
        print "Overburden is negative. Please check"
        exit()
    intvls_r = max(int(h*c / res), 1)  #number of intervals along the buffer radius, close enough
    res_r = h*c / intvls_r  #effective resolution along the radius
    buffer = np.array(point)  #initialize buffer, first item is exactly at station point

    # calculate local coordinates for grid along a ring and add to point coor
    for i in range(intvls_r):
        r = c*h - i*res_r
        perim = 2 * r * pi 
        intvls_c = max(int(perim/res), 1)  #number of intervals along a ring, close enough
        item = np.array([0.0, 0.0])  #initialize       
        for j in range(intvls_c):
            item[0] = (sin((2*pi) / intvls_c *(j+1)) *r) + point[0]
            item[1] = (cos((2*pi) / intvls_c *(j+1)) *r) + point[1]
            buffer = np.vstack((buffer, item))  #vstack works with arrays of diff nr of items, append does not        

    return buffer

In [23]:
# create csv file with all buffer points

# point = alignment_stationed_xy
# create alignment_stationed_xy from alignment_stationed_df with x,y of all station points
alignment_stationed_xy = alignment_stationed_df.as_matrix(columns=['x_align','y_align'])
# overburden = alignment_stationed_h
alignment_stationed_h = alignment_stationed_df.as_matrix(columns=['h'])

# initialize buffer_df, buffer_all_df, buffer_all
buffer_all = {}
buffer_df = pd.DataFrame(columns=["id_point", "x_align", "y_align", "z_align","h" ,"x_buffer", "y_buffer"])
buffer_all_df = pd.DataFrame(columns=["id_point", "x_align", "y_align", "z_align","h","x_buffer", "y_buffer"])

for n in range(0, len(alignment_stationed_df)): 
    buffer_point = make_buffer(point=alignment_stationed_xy[n], overburden=alignment_stationed_h[n])
    buffer_all[n] = buffer_df.copy(deep=False)  #copy of initialized buffer_df
    #print("n: ", n)
    #print(buffer_all)
    buffer_all[n]["id_point"] = [n] * len(buffer_point)  #list with len(buffer_point) number of n values) 
    buffer_all[n]["x_align"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("x_align")]] * len(buffer_point) )      
    buffer_all[n]["y_align"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("y_align")]] * len(buffer_point) )      
    buffer_all[n]["z_align"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("z_align")]] * len(buffer_point) )           
    buffer_all[n]["h"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("h")]] * len(buffer_point) )           
    buffer_all[n]["x_buffer"] = buffer_point[0:,0]
    buffer_all[n]["y_buffer"] = buffer_point[0:,1]
    buffer_all_df = pd.concat([buffer_all_df, buffer_all[n]])
    #print(buffer_all_df)
    
# add variable "id_buffer_point" to buffer_all_df
buffer_all_df = buffer_all_df.reset_index(drop=True)
buffer_all_df["id_buffer_point"] = buffer_all_df.index    

# save buffer_all_df to csv
buffer_all_df.to_csv(Buffer_all_csv, sep=",", na_rep="NaN")

buffer_all_df.head()
                                               #ToDo KLK: make check plot of alignment, station points, buffer_all 

,id_point,x_align,y_align,z_align,h,x_buffer,y_buffer,id_buffer_point
0,0,2612071.548,1269294.122,220.962,32.562,2.612072e+06,1.269294e+06,0
1,0,2612071.548,1269294.122,220.962,32.562,2.612087e+06,1.269299e+06,1
2,0,2612071.548,1269294.122,220.962,32.562,2.612081e+06,1.269281e+06,2
3,0,2612071.548,1269294.122,220.962,32.562,2.612062e+06,1.269281e+06,3
4,0,2612071.548,1269294.122,220.962,32.562,2.612056e+06,1.269299e+06,4


In [24]:
# add required fiedl "z_dtm_buffer" and calcualted "dist" to buffer_all_df 

# buffer_all_df to Buffer_all_shp                                   #ToDo JK: reuse df -> shp function from above
# list of shapely geometry points
buffer_all_geometry = ( 
    [sp.geometry.Point(row['x_buffer'], row['y_buffer']) for key, row in buffer_all_df.iterrows()] )
# create buffer_all_geometry_df
buffer_all_geometry_df = gpd.GeoDataFrame(buffer_all_df, geometry=buffer_all_geometry, crs = crs)
# write df to Buffer_all_shp
buffer_all_geometry_df.to_file(Buffer_all_shp, driver='ESRI Shapefile') 
#print(buffer_all_geometry_df.head())

# get DTM values for Buffer_all_shp                                #ToDo JK: reuse what.points function from above
#   write to Buffer_dtm_csv
processing.runalg("grass7:r.what.points",DTM,Buffer_all_shp, "NA",",", 500,True,False,False,False,False,
                  grass_region,-1,0.0001,Buffer_dtm_csv)

# create buffer_dtm_df (dataframe) from Buffer_dtm_csv
buffer_dtm_df = pd.read_csv(Buffer_dtm_csv)

# rename col=tmp... to "z_dtm_buffer"
buffer_dtm_df_col_tmp = [col for col in buffer_dtm_df.columns if 'tmp' in col]
if len(buffer_dtm_df_col_tmp) != 1:
    print "Extraction of DTM col=tmp did not work properly for buffer. Please check"
    exit()
buffer_dtm_df = buffer_dtm_df.rename(
    columns= {buffer_dtm_df_col_tmp[0]: "z_dtm_buffer"})
#print(buffer_dtm_df.head())

# write buffer_dtm_df["z_dtm"] to buffer_all_df["z_dtm"]
buffer_all_df["z_dtm_buffer"] = buffer_dtm_df["z_dtm_buffer"]
#print(buffer_all_df.head())

# calculate "dist" between each buffer point and associated alignment point 

#buffer_all_df["dist"] = buffer_all_df["z_dtm_buffer"] -buffer_all_df["z_align"]    #ToDo KLK: !!!Need 3-d distance

buffer_all_df["dist"] = (((buffer_all_df["x_align"]-buffer_all_df["x_buffer"])**2 + 
                         (buffer_all_df["y_align"]-buffer_all_df["y_buffer"]) **2) +
                         (buffer_all_df["h"])**2)**(0.5)
                         
# clean up
buffer_all_df = buffer_all_df.drop(columns =["geometry"])
buffer_all_df.head()

This can cause unexpected results.


,id_point,x_align,y_align,z_align,h,x_buffer,y_buffer,id_buffer_point,z_dtm_buffer,dist
0,0,2612071.548,1269294.122,220.962,32.562,2.612072e+06,1.269294e+06,0,253.5240,32.562000
1,0,2612071.548,1269294.122,220.962,32.562,2.612087e+06,1.269299e+06,1,252.5021,36.405423
2,0,2612071.548,1269294.122,220.962,32.562,2.612081e+06,1.269281e+06,2,252.6407,36.405423
3,0,2612071.548,1269294.122,220.962,32.562,2.612062e+06,1.269281e+06,3,254.1904,36.405423
4,0,2612071.548,1269294.122,220.962,32.562,2.612056e+06,1.269299e+06,4,253.7920,36.405423


In [25]:
# calculate minimum distance to terrain in each buffer ring
#   required output data: dist for each buffer point, min_dist for buffer ring


In [26]:
# calculate "min_dist"

buffer_all_df["min_dist"] = np.nan

for n in range(0, len(alignment_stationed_df)):
    buffer_all_df_sel = buffer_all_df.loc[(buffer_all_df["id_point"] == n),]
    dist_idxmin=buffer_all_df_sel['dist'].idxmin()
    buffer_all_df.loc[(buffer_all_df["id_buffer_point"] == dist_idxmin), "min_dist"] = "MIN"

buffer_all_df.to_csv(Buffer_all_csv, header=True, index=False)  #no header
buffer_all_df

buffer_all_df.loc[(buffer_all_df["min_dist"] == "MIN"),]

,id_point,x_align,y_align,z_align,h,x_buffer,y_buffer,id_buffer_point,z_dtm_buffer,dist,min_dist
0,0,2.612072e+06,1.269294e+06,220.962000,32.562000,2.612072e+06,1.269294e+06,0,253.5240,32.562000,MIN
6,1,2.612111e+06,1.269265e+06,218.972667,38.415633,2.612111e+06,1.269265e+06,6,257.3883,38.415633,MIN
13,2,2.612150e+06,1.269235e+06,216.983333,40.087367,2.612150e+06,1.269235e+06,13,257.0707,40.087367,MIN
20,3,2.612190e+06,1.269206e+06,214.994000,41.362500,2.612190e+06,1.269206e+06,20,256.3565,41.362500,MIN
27,4,2.612229e+06,1.269177e+06,213.004667,42.799433,2.612229e+06,1.269177e+06,27,255.8041,42.799433,MIN
34,5,2.612269e+06,1.269147e+06,211.015333,44.962267,2.612269e+06,1.269147e+06,34,255.9776,44.962267,MIN
42,6,2.612308e+06,1.269118e+06,209.026000,48.532500,2.612308e+06,1.269118e+06,42,257.5585,48.532500,MIN
50,7,2.612347e+06,1.269088e+06,207.036667,49.593733,2.612347e+06,1.269088e+06,50,256.6304,49.593733,MIN
58,8,2.612387e+06,1.269059e+06,205.047333,56.168267,2.612387e+06,1.269059e+06,58,261.2156,56.168267,MIN
67,9,2.612426e+06,1.269030e+06,203.058000,59.661100,2.612426e+06,1.269030e+06,67,262.7191,59.661100,MIN


In [27]:
# calculate hydraulic confinement safety factor at each station point
#   required input data: reference maximum water pressure elevation (static or dynamic ??) 





In [28]:
# plot results for hydraulic confinement safety factor as horizontal bar beneath longitudinal profile 





In [29]:
alignment_df.head()

,Point,Type,Station,Northing,Easting,Elevation,distance_intermed_align
0,0,NaN,204+874.1,1269294.122,2612071.548,220.962,442.544958
1,221,PT,205+389.861,1269029.594,2612426.331,203.058,3736.888722
2,EC10:3,EC,209+528.128,1265960.779,2614558.633,218.579,NaN


In [30]:
alignment_stationed_df.head()

,x_align,y_align,id_point,distance_stat,distance_stat_sum,Station,z_align,z_dtm_align,h
0,2.612072e+06,1269294.122,0,49.171662,49.171662,204+874.1,220.962000,253.5240,32.562000
1,2.612111e+06,1269264.730,1,49.171662,98.343324,NaN,218.972667,257.3883,38.415633
2,2.612150e+06,1269235.338,2,49.171662,147.514986,NaN,216.983333,257.0707,40.087367
3,2.612190e+06,1269205.946,3,49.171662,196.686648,NaN,214.994000,256.3565,41.362500
4,2.612229e+06,1269176.554,4,49.171662,245.858310,NaN,213.004667,255.8041,42.799433


In [31]:
buffer_all_df.head()

,id_point,x_align,y_align,z_align,h,x_buffer,y_buffer,id_buffer_point,z_dtm_buffer,dist,min_dist
0,0,2612071.548,1269294.122,220.962,32.562,2.612072e+06,1.269294e+06,0,253.5240,32.562000,MIN
1,0,2612071.548,1269294.122,220.962,32.562,2.612087e+06,1.269299e+06,1,252.5021,36.405423,NaN
2,0,2612071.548,1269294.122,220.962,32.562,2.612081e+06,1.269281e+06,2,252.6407,36.405423,NaN
3,0,2612071.548,1269294.122,220.962,32.562,2.612062e+06,1.269281e+06,3,254.1904,36.405423,NaN
4,0,2612071.548,1269294.122,220.962,32.562,2.612056e+06,1.269299e+06,4,253.7920,36.405423,NaN


In [100]:
z_align = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_align'].tolist(),
    name = "z_align",
    mode='lines',
    line=dict(width=0.5,
              color='rgb(192, 192, 192)'),
    fill='tonexty'
)

z_dtm_align = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_dtm_align'].tolist(),
    name = "z_dtm_align",
    mode='lines',
    line=dict(width=0.5,
              color='rgb(196, 97, 26)'),
    fill='tonexty'
)

data = [z_align, z_dtm_align]

layout = go.Layout(
    title='Longitudinal Section',
    showlegend=True,
    xaxis=dict(
        title= "Stationing",
    ),
    yaxis=dict(
        title = "m a.s.l."
    )
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-area-plot')

In [116]:
# z_align = go.Scatter(
#     x=alignment_stationed_df['distance_stat_sum'].tolist(),
#     y=alignment_stationed_df['z_align'].tolist(),
#     name = "z_align",
#     mode='lines',
#     line=dict(width=0.5,
#               color='rgb(196, 97, 26)'),
#     fill='tonexty'
# )

z_dtm_align = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_dtm_align'].tolist(),
    name = "Terrain",
    mode='lines',
    line=dict(width=0.5,
              color='rgb(196, 97, 26)'),
    fill='tonexty'
)

z_align_line = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_align'].tolist(),
    line=dict(width=3,
              color='rgb(0, 0, 0)'),
    name ="Tunnel"
)

data = [z_dtm_align, z_align_line]

layout = go.Layout(
    title='Longitudinal Section',
    showlegend=True,
    xaxis=dict(
        title= "Stationing",
    ),
    yaxis=dict(
        title = "m a.s.l."
    )
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-area-plot')